# Task 0

Ingest Sakila (SQLite → DuckDB)

This section attaches the SQLite Sakila DB, copies all tables into a **DuckDB** file, and verifies the result with a few checks.

In [98]:
import duckdb

# "r" - read mode for the file we open
with open("../sql/load_sakila.sql", "r") as sql_file:
    loading_script = sql_file.read()

print(loading_script)

INSTALL sqlite;

LOAD sqlite;

CALL sqlite_attach ('../data/sqlite-sakila.db');


In [99]:
type(loading_script)

str

### Connect to **DuckDB**

In [100]:
from pathlib import Path

duckdb_path = "../data/sakila.duckdb"

# throws away sakila.duckdb if it exists -> makes it idempotent
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn:
    conn.sql(loading_script)

    description = conn.sql("desc;").df()
    films = conn.sql("FROM film;").df()
    actors = conn.sql("FROM actor;").df()


description

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
3,sakila,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",False
4,sakila,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
5,sakila,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
6,sakila,main,customer_list,"[ID, name, address, zip_code, phone, city, cou...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False
7,sakila,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False
8,sakila,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False
9,sakila,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False


In [101]:
films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   film_id               1000 non-null   int64         
 1   title                 1000 non-null   object        
 2   description           1000 non-null   object        
 3   release_year          1000 non-null   object        
 4   language_id           1000 non-null   int64         
 5   original_language_id  0 non-null      Int64         
 6   rental_duration       1000 non-null   int64         
 7   rental_rate           1000 non-null   float64       
 8   length                1000 non-null   int64         
 9   replacement_cost      1000 non-null   float64       
 10  rating                1000 non-null   object        
 11  special_features      1000 non-null   object        
 12  last_update           1000 non-null   datetime64[us]
dtypes: Int64(1), dateti

In [102]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     200 non-null    float64       
 1   first_name   200 non-null    object        
 2   last_name    200 non-null    object        
 3   last_update  200 non-null    datetime64[us]
dtypes: datetime64[us](1), float64(1), object(2)
memory usage: 6.4+ KB


In [103]:
films.columns

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update'],
      dtype='object')

In [104]:
actors.columns

Index(['actor_id', 'first_name', 'last_name', 'last_update'], dtype='object')

In [105]:
films.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [106]:
actors.head(2)

,actor_id,first_name,last_name,last_update
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59


# Task 1

Explore and analyze some questions.

### a) — Films longer than 180 minutes
We want to list all films in the Sakila database that have a runtime (length) greater than 180 minutes.

In [107]:
query_sakila("""
    SELECT
        film_id,
        title,
        length
    FROM film
    WHERE length > 180
    ORDER BY length DESC
    LIMIT 5; 
""") # total was 38 films longer than 180 minutes

,film_id,title,length
0,182,CONTROL ANTHEM,185
1,141,CHICAGO NORTH,185
2,349,GANGS PRIDE,185
3,212,DARN FORRESTER,185
4,426,HOME PITY,185


### b) — Films with "love" in the title
We search for all films where the title contains the word **"love"** (not case-sensitive). Sort the results alphabetically and show a small preview.

In [108]:
query_sakila("""
    SELECT
        film_id,
        title,
        length
    FROM film
    WHERE LOWER(title) LIKE '%love%'
    ORDER BY title
    LIMIT 5;
""")

,film_id,title,length
0,374,GRAFFITI LOVE,117
1,448,IDAHO LOVE,172
2,449,IDENTITY LOVER,119
3,458,INDIAN LOVE,135
4,511,LAWRENCE LOVE,175


In [109]:
query_sakila("""
    SELECT
        film_id,
        title,
        length
    FROM film
    WHERE LOWER(title) ~ '[[:<:]]love[[:>:]]'
    ORDER BY title
    LIMIT 5;
""")

InvalidInputException: Invalid Input Error: invalid character class range: [:<:]